In [1]:
from collections import defaultdict
from itertools import chain
import networkx as nx

In [2]:
def read_movement_instructions(instruction_row: str) -> list[tuple[str, int]]:
    instruction_row = instruction_row.rstrip()
    if instruction_row[-1] in ["R", "L"]:
        instruction_row = instruction_row + "0"
    if instruction_row[0] not in ["R", "L"]:
        instruction_row = "R" + instruction_row
    char_lst = []
    int_lst = []
    last_char = None
    for idx, entry in enumerate(instruction_row):
        if entry in ["L", "R"]:
            char_lst.append(entry)
            if last_char is not None:
                int_lst.append(int(instruction_row[last_char + 1 : idx]))
            last_char = idx
    int_lst.append(int(instruction_row[last_char + 1 :]))

    assert len(char_lst) == len(int_lst)
    return list(zip(char_lst, int_lst))

In [3]:
def parse_input(filename: str) -> tuple[nx.Graph, list[tuple[str, int]]]:
    graph = nx.Graph()

    def _add_neighbor(u_idx, v_idx, key: str) -> bool:
        if not graph.has_node(v_idx) or not graph.has_node(u_idx):
            return False

        if graph.nodes[u_idx]["passable"] and graph.nodes[v_idx]["passable"]:
            graph.add_edge(u_idx, v_idx, direction=key)
        return True

    boundary_left = defaultdict(list)
    boundary_top = defaultdict(list)
    with open(filename) as f:
        for row_idx, row in enumerate(f):
            if not row.rstrip():
                for col_idx, col_lst in boundary_top.items():
                    for lowest_row_idx in range(row_idx - 1, -1, -1):
                        if graph.has_node((lowest_row_idx, col_idx)):
                            _add_neighbor(
                                (lowest_row_idx, col_idx), col_lst[-1], "vertical"
                            )
                            break

                movement_instructions = read_movement_instructions(next(f))
                break

            for col_idx, char in enumerate(row):
                if char == " " or char == "\n":
                    if boundary_top[col_idx]:
                        _add_neighbor(
                            (row_idx - 1, col_idx),
                            boundary_top[col_idx][-1],
                            "vertical",
                        )

                    if boundary_left[row_idx]:
                        _add_neighbor(
                            (row_idx, col_idx - 1),
                            boundary_left[row_idx][-1],
                            "horizontal",
                        )
                    continue

                graph.add_node(
                    (row_idx, col_idx),
                    passable=(char != "#"),
                )

                if not _add_neighbor(
                    (row_idx, col_idx), (row_idx, col_idx - 1), "horizontal"
                ):
                    boundary_left[row_idx].append((row_idx, col_idx))
                if not _add_neighbor(
                    (row_idx, col_idx), (row_idx - 1, col_idx), "vertical"
                ):
                    boundary_top[col_idx].append((row_idx, col_idx))

        return graph, movement_instructions

In [4]:
def starting_pos(graph: nx.Graph) -> tuple[tuple[int, int], int]:
    col_idx = 0
    while True:
        if graph.has_node((0, col_idx)):
            if graph.nodes[(0, col_idx)]["passable"]:
                return ((0, col_idx), 3)
        else:
            col_idx += 1

In [5]:
def graph_step(graph: nx.Graph, pos: tuple[int, int], dir: int) -> tuple[int, int]:
    dir_strs = {
        0: "horizontal",
        1: "vertical",
        2: "horizontal",
        3: "vertical",
    }
    row_steps = {
        0: 0,
        1: 1,
        2: 0,
        3: -1,
    }
    col_steps = {
        0: 1,
        1: 0,
        2: -1,
        3: 0,
    }

    new_pos = (pos[0] + row_steps[dir], pos[1] + col_steps[dir])

    if graph.has_edge(pos, new_pos):
        return new_pos

    if graph.has_node(new_pos):
        return pos

    # we have to wrap
    for u_idx, v_idx, direction in graph.edges(pos, "direction"):
        if direction != dir_strs[dir]:
            continue
        if abs(u_idx[0] - v_idx[0]) + abs(u_idx[1] - v_idx[1]) > 1:
            ret_pos = v_idx if u_idx == pos else u_idx
            return ret_pos
    return pos

In [6]:
def task_one(
    graph: nx.Graph, movement_instructions: list[tuple[str, int]], verbose: bool = False
) -> tuple[int, nx.Graph, list[tuple[tuple[int, int], int]]]:
    pos, dir = starting_pos(graph)
    path_lst = []
    for instr, amount in movement_instructions:
        if instr == "L":
            dir = (dir - 1) % 4
        elif instr == "R":
            dir = (dir + 1) % 4
        else:
            raise AssertionError(dir)
        if verbose:
            print(f"Instruction {instr} {amount}: Now at {pos} facing {dir}")

        for _step in range(amount):
            new_pos = graph_step(graph, pos, dir)
            if new_pos != pos:
                path_lst.append((pos, dir))
                if verbose:
                    print(f"\tStep {_step}: from {pos} to {new_pos}")
            pos = new_pos

    path_lst.append((pos, dir))
    return 1000 * (pos[0] + 1) + 4 * (pos[1] + 1) + dir, graph, path_lst

In [7]:
val, graph, path_lst = task_one(*parse_input("test-input.txt"))
print(val, path_lst[-1])

6032 ((5, 7), 0)


In [8]:
val, graph, path_lst = task_one(*parse_input("input.txt"))
print(val, path_lst[-1])

43466 ((42, 115), 2)


In [9]:
def print_maze(graph: nx.Graph, path_lst: list[tuple[tuple[int, int], int]]):
    ret_str = defaultdict(str)
    next_lines = defaultdict(str)
    max_row_idx = max(node[0] for node in graph.nodes)
    max_col_idx = max(node[1] for node in graph.nodes)
    for row_idx in range(-1, max_row_idx + 1):
        for col_idx in range(-1, max_col_idx + 2):
            if graph.has_node((row_idx, col_idx)):
                ret_str[row_idx] += (
                    "o" if graph.nodes[(row_idx, col_idx)]["passable"] else "#"
                )
                if graph.has_edge((row_idx, col_idx), (row_idx, col_idx + 1)):
                    ret_str[row_idx] += "–"
                else:
                    ret_str[row_idx] += " "
                if graph.has_edge((row_idx, col_idx), (row_idx + 1, col_idx)):
                    next_lines[row_idx] += "| "
                elif not graph.has_node((row_idx + 1, col_idx)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx), "direction")
                    if dir == "vertical" and (row_idx - 1, col_idx) not in [u, v]
                ]:
                    next_lines[row_idx] += "| "
                else:
                    next_lines[row_idx] += "  "
            else:
                if graph.has_node((row_idx, col_idx - 1)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx - 1), "direction")
                    if dir == "horizontal" and (row_idx, col_idx - 2) not in [u, v]
                ]:
                    ret_str[row_idx] += "– "
                elif graph.has_node((row_idx, col_idx + 1)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx + 1), "direction")
                    if dir == "horizontal" and (row_idx, col_idx + 2) not in [u, v]
                ]:
                    ret_str[row_idx] += " –"
                else:
                    ret_str[row_idx] += "  "

                if graph.has_node((row_idx + 1, col_idx)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx + 1, col_idx), "direction")
                    if dir == "vertical" and (row_idx + 2, col_idx) not in [u, v]
                ]:
                    next_lines[row_idx] += "| "
                else:
                    next_lines[row_idx] += "  "

    dir_chars = {
        0: ">",
        1: "v",
        2: "<",
        3: "^",
    }

    for pos, dir in path_lst:
        tmp = ret_str[pos[0]] if dir in [0, 2] else next_lines[pos[0]]

        if dir == 0:
            ret_str[pos[0]] = (
                tmp[: 2 * pos[1] + 3] + dir_chars[dir] + tmp[3 + 2 * (pos[1]) + 1 :]
            )
        if dir == 1:
            next_lines[pos[0]] = (
                tmp[: 2 * pos[1] + 2]
                + dir_chars[dir]
                + " "
                + tmp[2 + 2 * (pos[1] + 1) :]
            )
        elif dir == 2:
            ret_str[pos[0]] = (
                tmp[: 2 * pos[1] + 1] + dir_chars[dir] + tmp[1 + 2 * pos[1] + 1 :]
            )
        elif dir == 3:
            next_lines[pos[0] - 1] = (
                tmp[: 2 * pos[1] + 2]
                + dir_chars[dir]
                + " "
                + tmp[2 + 2 * (pos[1] + 1) :]
            )

    return "\n".join(
        chain.from_iterable(
            [ret_str[idx], next_lines[idx]] for idx in sorted(ret_str.keys())
        )
    )

In [10]:
graph, movement_instructions = parse_input("test-input.txt")
val, graph, path_lst = task_one(graph, movement_instructions)
print(print_maze(graph, path_lst))

                                    
                  | | |             
                  o>o>o #           
                  |   v             
                 –o # o–o –         
                      v |           
                  # o–o–o           
                    | v |           
                 –o–o–o–o –         
  | | |   | | | | | | v             
  o–o–o # o–o–o–o–o–o–o #           
  | | |   | | | v   | v             
 –o>o>o>o–o–o–o–o># o–o>o>–         
  | |   v | | |     | | |           
 –o–o # o–o–o–o # o–o–o–o –         
  | |   v | | |   | |   |           
 –o–o–o–o>o>o>o>o–o–o # o –         
  | | |   | | | v | |     | |   |   
                 –o–o–o # o–o–o–o – 
                  | | |   |   | |   
                 –o–o–o–o–o # o–o – 
                  |   | | |   | |   
                 –o # o–o–o–o–o–o – 
                  |   | | | |   |   
                 –o–o–o–o–o–o # o – 
                  | | |   | |   |   
